In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\U.B.A Yadav\OneDrive\Desktop\songdata.csv")

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.shape

(57650, 4)

In [5]:
df = df.sample(n = 5000).drop('link' , axis = 1).reset_index(drop = True)

In [6]:
df.shape

(5000, 3)

In [7]:
df.head()

,artist,song,text
0,Utopia,Too Much Water,"Too much, it's too much, too much \nGot too m..."
1,John Waite,White Lightening,When I was young \nThey took me to the dentis...
2,Who,Here 'tis,"Whoa, whoa, whoa (whoa, whoa, whoa), \nI said..."
3,Cat Stevens,Baby Get Your Head Screwed On,"Well if you're feelin' low, take it slow \n ..."
4,Reba Mcentire,If You Only Knew,So here you are again \nStanding at my door ...


In [8]:
df['text'] = df['text'].str.lower().replace(r'[^a-zA-Z0-9]', ' ', regex=True)

In [9]:
df['text'][0]

'too much  it s too much  too much   got too much water under the bridge   got too much  too much  got too much   got too much water under the bridge   i got to think of something  think of something   i got to think of something   we got to hang together  we got to hang together   we got to hang together  or hang separately   maybe we got time to burn   i got a yen to hear myself talk   but i don t want to make that kind of history   put your hand on the rock   and maybe you ve got nothing left to learn   you put me in a state   but do you want to make that kind of history   put your hand on the rock   and let it all run out   you know why we got to hang together   number one ain t always number one   instant karma s always coming back   and i don t want to make that kind of history   put your hand on the rock   tell ya mamma nature s on the run   bad karma s running in the back   but do we want to make that kind of history   put your hand on the rock   and let it all run out   you kn

In [10]:
import nltk
from nltk.stem.porter import PorterStemmer

# Initialize the Porter Stemmer
ps = PorterStemmer()

def tokenization_and_stemming(txt):
    # Tokenize the text
    tokens = nltk.word_tokenize(txt)
    
    # Initialize an empty list for storing stemmed words
    stemming = []

    # Apply stemming to each word in the tokenized text
    for w in tokens:
        stemming.append(ps.stem(w))
    
    # Join the stemmed words to form the processed text
    return " ".join(stemming)

In [11]:
# Apply the tokenization_and_stemming function to the 'text' column
df['text'] = df['text'].apply(lambda x: tokenization_and_stemming(x))

# Display the DataFrame with processed 'text' column
print(df)

                 artist                           song  \
0                Utopia                 Too Much Water   
1            John Waite               White Lightening   
2                   Who                      Here 'tis   
3           Cat Stevens  Baby Get Your Head Screwed On   
4         Reba Mcentire               If You Only Knew   
...                 ...                            ...   
4995  The White Stripes       In The Cold, Cold, Night   
4996          Zebrahead                           Blur   
4997            Rihanna                          Vogue   
4998            Misfits         Dr. Phibes Rises Again   
4999      Glen Campbell                    Without Her   

                                                   text  
0     too much it s too much too much got too much w...  
1     when i wa young they took me to the dentist i ...  
2     whoa whoa whoa whoa whoa whoa i said whoa whoa...  
3     well if you re feelin low take it slow babi ge...  
4     so here

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

# Initialize the Porter Stemmer
ps = PorterStemmer()

def tokenization(txt):
    # Tokenize the text
    tokens = nltk.word_tokenize(txt)
    
    # Apply stemming using list comprehension
    stemming = [ps.stem(w) for w in tokens]
    
    # Join the stemmed words to form the processed text
    return " ".join(stemming)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
tfid = TfidfVectorizer(stop_words = 'english')
matrix = tfid.fit_transform(df['text'])

In [15]:
matrix.shape

(5000, 17621)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'matrix' is a valid matrix or array containing vectorized representations

similarity = cosine_similarity(matrix)

In [21]:
similarity[0]


array([1.        , 0.05371719, 0.00518807, ..., 0.03589877, 0.01751007,
       0.00840383])

In [23]:
df['song'][0]

'Too Much Water'

In [26]:
df[df['song']== 'Too Much Water']

,artist,song,text
0,Utopia,Too Much Water,too much it s too much too much got too much w...


In [31]:
distances = sorted(list(enumerate(similarity[0])), reverse = False , key = lambda x :x[1])

In [47]:
def recommendation(song):
    song_matches = df[df['song'] == song]
    
    if not song_matches.empty:
        idx = song_matches.index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=False, key=lambda x: x[1])

        songs = []
        for i in distances[1:21]:
            songs.append(df.iloc[i[0]].song)

        return songs
    else:
        return f"No matches found for the song: {song}"


In [49]:
recommendation('All For Love')

['Malfunction',
 'Merry Christmas, Darling',
 'I Am The Walrus',
 "Baby, I'm A Big Star Now",
 'Ships In The Sky',
 'When A Child Is Born',
 "Bop' Til You Drop",
 'Los Bastardos',
 'Je Ne Veux Pas',
 'No Respect',
 'Walk Right In',
 'Pretending To See The Future',
 'Two Steppin Mind',
 'Ave Maria',
 'Steamrock Fever',
 'Help Me Make It Through The Night',
 'What Is That You Say',
 'This Old Dog',
 "Ain't Nobody's Business If I Do",
 'Make A Move']